In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
idg = ImageDataGenerator()

In [20]:
train_generator = idg.flow_from_directory('data/', 
                                          target_size = (256, 256), 
                                          batch_size = 16, 
                                          class_mode = None)

Found 37692 images belonging to 2 classes.


In [22]:
type(train_generator)

keras.preprocessing.image.DirectoryIterator

In [23]:
batch = train_generator.next()